In [ ]:
%pip install -U torch transformers typing-extensions

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openchat/openchat_3.5")
model = AutoModelForCausalLM.from_pretrained("openchat/openchat_3.5")

In [ ]:
print(tokenizer)

In [ ]:
import pandas as pd

data = pd.read_csv("/Workspace/Users/degarg@ucsd.edu/train.csv")
data = data[data['y'] == 1]
print(data.shape)

count = 30

In [ ]:
diag1 = []
for i in range(data.shape[0])[:count]:
    record = data.iloc[i]['X']
    input_text = f"""Following is a discharge summary of a patient within three back ticks. \
    Given the note, what is the cancer diagnosis for the patient? ```{record}```\
    """
    input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"]

    generated_ids = model.generate(input_ids, max_new_tokens=40, do_sample=True, temperature=0.7)
    generated_text = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
    diag1.append(generated_text.replace(input_text, ""))

In [ ]:
diag2 = []
for i in range(data.shape[0])[:count]:
    record = data.iloc[i]['X']
    input_text = f"""Following is a discharge summary of a patient within three back ticks. \
    Given the note, what stage cancer is the patient diagnosed with? ```{record}```\
    """
    input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"]

    generated_ids = model.generate(input_ids, max_new_tokens=30, do_sample=True, temperature=0.7)
    generated_text = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
    diag2.append(generated_text.replace(input_text, ""))

In [ ]:
adm = []
for i in range(data.shape[0])[:count]:
    record = data.iloc[i]['X']
    input_text = f"""Following is a discharge summary of a patient within three back ticks. \
    Given the note, why was the patient admitted? ```{record}```\
    """
    input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"]

    generated_ids = model.generate(input_ids, max_new_tokens=80, do_sample=True, temperature=0.65)
    generated_text = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
    adm.append(generated_text.replace(input_text, ""))

In [ ]:
medication = []
for i in range(data.shape[0])[:count]:
    record = data.iloc[i]['X']
    input_text = f"""Following is a discharge summary of a patient within three back ticks. \
        Given the note, what is the medication given to the patient chronologically? ```{record}```\
        """
    input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"]

    generated_ids = model.generate(input_ids, max_new_tokens=80, do_sample=True, temperature=0.25)
    generated_text = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
    medication.append(generated_text.replace(input_text, ""))

In [ ]:
# store the responses in the dataframe

res = data[:count]
res['medication'] = medication
res['type'] = diag1
res['stage'] = diag2
res['admission_reason'] = adm

In [ ]:
from datetime import datetime

today = datetime.now()
res.to_csv(f"/Workspace/Users/degarg@ucsd.edu/results_{today}.csv", index=False)